# Robust Matrix Completion by Riemannian Optimization
by Ilya Zharikov

In [3]:
import numpy as np
import sys
from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.linalg import svds, norm, eigs
from skimage import io, color
import matplotlib.pyplot as plt
%matplotlib inline
import time
sys.path.append('./../')
from data import *
from errors import *

In [25]:
def get_f(delta, X, omega, vec_data, lam):
    X_vec = get_sampling_vector(X, omega)
    return (np.sum(np.sqrt(delta**2 + (X_vec - vec_data)**2)) 
            + lam * (np.linalg.norm(X, 'fro') - np.linalg.norm(X_vec)))

def get_S(delta, X_vec, omega, vec_data, lam, dims):
    S = np.zeros(omega.shape[0])
    S = (X_vec - vec_data) /  np.sqrt(delta**2 + (X_vec - vec_data)**2) - 2*lam*X_vec
    return reconstruct_matrix(S, omega, dims)

def get_project_UEV(U, E, V, S, lam):
    U_ = S.dot(V) - U.dot(U.T.dot(S.dot(V)))
    E_ = U.T.dot(S.dot(V)) + 2*lam*E
    V_ = S.T.dot(U) - V.dot(V.T.dot(S.T.dot(U)))
    return U_, E_, V_

def grad_f(delta, X, omega, vec_data, lam, dims):
    U, E, V = np.linalg.svd(X)
    V = V.T
    E = np.diag(E)
    X_vec = get_sampling_vector(X, omega)
    S = get_S(delta, X_vec, omega, vec_data, lam, dims)
    U_, E_, V_ = get_project_UEV(U, E, V, S, lam)
    return U.dot(E_.dot(V.T)) + U_.dot(V.T) + U.dot(V_.T)

def RG(delta, X, omega, vec_data, lam, dims, tol):
    tau = 0.05
    cnt = 1
    X_pred = X.copy()
    eps = np.inf
    while (eps > tol*1e-1) & (cnt < 1e4):
        grad = grad_f(delta, X_pred, omega, vec_data, lam, dims)
        X_next = X_pred - tau*grad
        #tau = 1. / cnt
        #tau *= 0.9
        eps = RMSE(X_pred, X_next)
        X_pred = X_next.copy()
        cnt += 1
    return X_next

def RMC(omega, vec_data, dims, X_0, delta_0, theta, tol, lam):
    f_pred = np.inf
    k = 0
    delta_pred = delta_0
    X_pred = X_0.copy()
    eps = np.inf
    while (eps > tol):
        X_opt = X_pred.copy()
        X_next = RG(delta_pred, X_pred, omega, vec_data, lam, dims, tol)
        f_next = get_f(delta_pred, X_next, omega, vec_data, lam)
        eps = f_pred - f_next
        delta_next = theta * delta_pred
        
        f_pred = f_next
        X_pred = X_next.copy()
        delta_pred = delta_next
        print eps, tol
        
    return X_opt

In [29]:
seed = 1
rank = 5
dims = (100, 100)
nsamp = int(np.ceil(0.1 * np.prod(dims)))

omega = sampling_operator(nsamp, dims, seed)
M = get_matrix(rank, dims)
data = get_sampling_vector(M, omega)

max_iter = 1e4
tol = 1e-3
lam = 1e-4
theta = 0.1
delta_0 = 1.
X_0 = reconstruct_matrix(data, omega, dims)
U, S, V = np.linalg.svd(X_0)
U = U[:, :rank]
S = S[:rank]
V = V[:rank, :]
X_0 = U.dot(np.diag(S).dot(V))

t0 = time.time()
X_opt = RMC(omega, data, dims, X_0, delta_0, theta, tol, lam)
rmse_error = RMSE(M, X_opt)
t1 = time.time()

print 'REPORT:'
print '                   RMSE: %.5f;' % rmse_error
print '                   Time: %d min %d sec.' % (int(t1 - t0) // 60, int(t1 - t0) % 60)

inf 0.001
900.016662094 0.001
75.0004613768 0.001
4.40034080924e-05 0.001
REPORT:
                   RMSE: 2.02270;
                   Time: 1 min 33 sec.
